
## RAGモデルの作成

### Databricks Foundation Model APIの設定

Databricks Foundation Model API (FMAPI) を使用してLLMを呼び出します。

In [0]:
# MLflowとDatabricks SDKのインストール
%pip install mlflow databricks-sdk databricks-agents databricks-langchain
dbutils.library.restartPython()

In [0]:
%run ./00_config

### LLMやRAGにおけるクエリとプロンプトの違い

[デイリーライフAI｜RAG（Retrieval Augmented Generation）のプロセスとその応用：プロンプトとクエリの違い](https://daily-life-ai.com/113/)より

> プロンプトはユーザーがAIに提供する文脈や指示で、AIが生成する内容の方向性を決定します。
> 
> 一方、クエリは関連情報を検索するための具体的な質問やキーワードです。

ユーザーからの質問（query）
```
"福永敦史の卒論の目的はなんですか？"
```

プロンプト
```
あなたはテキスト整理のプロです。
以下は参考情報です:

{PDFのドキュメント情報}

質問: この卒論の目的は何ですか？

参考情報に基づいてわかりやすく回答してください。
```

ユーザーからの質問 = queryでもあるし prompt の一部でもある

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import ChatMessage, ChatMessageRole
from databricks.vector_search.client import VectorSearchClient

# WorkspaceClientの初期化
w = WorkspaceClient()


def query_foundation_model(
    prompt: str, endpoint: str = "databricks-gpt-oss-20b"
) -> str:
    """
    Databricks Foundation Model APIを呼び出す

    Args:
        prompt: LLMへの入力プロンプト
        endpoint: Foundation Modelのエンドポイント名

    Returns:
        LLMからの応答テキスト
    """
    response = w.serving_endpoints.query(
        name=endpoint, messages=[ChatMessage(role=ChatMessageRole.USER, content=prompt)]
    )
    return response.choices[0].message.content


## Auto-tracingの有効化し、チャット履歴を残す


### Auto-tracingが有効でない場合

In [0]:
# ベクトル検索インデックスの取得
vs_client = VectorSearchClient()
vector_index = vs_client.get_index(index_name=vector_index_path, endpoint_name="mydocs")


def retrieve_documents(query: str, num_results: int = 3) -> list:
    """
    ドキュメント検索
    """
    documents = vector_index.similarity_search(
        query_text=query, columns=["text"], num_results=num_results
    )

    return documents


def generate_answer(
    query: str,
    documents: list = None,
) -> tuple:
    """
    RAG (Retrieval Augmented Generation) で質問に回答する
    """
    # Augmentation: 検索結果＋質問をプロンプトとして整形
    prompt = (
        f"以下は参考情報です:\n{documents}\n\n"
        f"質問: {query}\n"
        "参考情報をもとに、わかりやすく日本語で回答してください。"
    )

    # Generation: 基盤モデルAPIで回答生成
    answer = query_foundation_model(prompt)
    return answer


def rag_pipeline(query: str) -> dict:
    """
    RAGパイプライン全体
    """
    # ステップ1: ドキュメント検索
    documents = retrieve_documents(query)

    # ステップ2: 回答生成
    answer = generate_answer(query=query, documents=documents)

    return {"query": query, "documents": documents, "answer": answer}


# 例: 質問に対してRAGで回答
user_query = "福永の卒論の目的は何ですか？"
answer = rag_pipeline(user_query)
print("=== トレーシングなしの実行 ===")
print(f"質問: {user_query}")
print("【回答】", answer)
print("\n⚠️ 実行履歴が残りません！")

### MLflowの**Auto-tracing**を使うと、
LLM呼び出しを自動的にトレースできます。

**span_type（スパンタイプ）**を設定することで、トレース画面に適切なアイコンが表示されます：
- `LLM`: LLM呼び出し（💬アイコン）
- `RETRIEVER`: ドキュメント検索（🔍アイコン）
- `CHAIN`: 複数ステップの処理（🔗アイコン）
- `TOOL`: ツール呼び出し（🔧アイコン）

In [0]:
import mlflow
import mlflow.openai

# Auto-tracingを有効化（OpenAI互換API用）
mlflow.openai.autolog()


@mlflow.trace(name="retrieve_documents", span_type="RETRIEVER")
def retrieve_documents(query: str, num_results: int = 3) -> list:
    """
    ドキュメント検索
    """
    documents = vector_index.similarity_search(
        query_text=query, columns=["text"], num_results=num_results
    )

    return documents


@mlflow.trace(name="generate_answer", span_type="LLM")
def generate_answer(
    query: str,
    documents: list = None,
) -> tuple:
    """
    RAG (Retrieval Augmented Generation) で質問に回答する

    Args:
        query: ユーザーの質問（文字列）
        vector_index: ベクトル検索インデックス（事前に作成済みのオブジェクト）
        llm_endpoint: FMAPIのエンドポイント名
        top_k: 検索する関連ドキュメント数

    Returns:
        answer: LLMによる回答
        documents: 検索で取得した関連ドキュメント（リスト）
    """
    # Augmentation: 検索結果＋質問をプロンプトとして整形
    prompt = (
        f"以下は参考情報です:\n{documents}\n\n"
        f"質問: {query}\n"
        "参考情報をもとに、わかりやすく日本語で回答してください。"
    )

    # Generation: 基盤モデルAPIで回答生成
    answer = query_foundation_model(prompt)
    return answer


@mlflow.trace(name="rag_pipeline", span_type="CHAIN")
def rag_pipeline(query: str) -> dict:
    """
    RAGパイプライン全体
    """
    # ステップ1: ドキュメント検索
    documents = retrieve_documents(query)

    # ステップ2: 回答生成
    answer = generate_answer(query=query, documents=documents)

    return {"query": query, "documents": documents, "answer": answer}


# トレーシングありで実行
print("=== トレーシングありの実行 ===")
answer = rag_pipeline("福永の卒論の目的は何ですか？")
print(f"回答: {answer}")
print("\n✅ トレースが記録されました！")

### トレーシング機能の具体的なユースケース

MLflow Trace ID`tr-198b4fc9b00f3a8e78106568de841836`で  
`generate_answer`を確認すると、以下のように、PDF情報を読み取って回答できていないことが判明。  

```
卒論の目的を正しくお答えするには、実際の論文の要旨・序論・研究課題の記載などの具体的な情報が必要です。  
現在「参考情報」が示されていないため、目的を特定することができません。 
```

`retrieve_documents`処理では問題なくドキュメント取得できていることが確認できたため、  
原因は回答を生成する`generate_answer`処理にあると推定。  

`generate_answer`処理を確認し、プログラムに問題があったため、修正し解決した。
